<a href="https://colab.research.google.com/github/FerBarrey/An-lisis-cuantitativo-avanzados/blob/main/Ejercicio_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 7

## Cargar la base

La siguiente base describe datos sobre los salarios, la formación y la experiencia de empleados que realizan entrevistas de trabajo.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import userdata
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {
    'username': userdata.get('KAGGLE_USER'),
    'key': userdata.get('KAGGLE_KEY')}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mohithsairamreddy/salary-data

Dataset URL: https://www.kaggle.com/datasets/mohithsairamreddy/salary-data
License(s): Community Data License Agreement - Sharing - Version 1.0
  0% 0.00/16.6k [00:00<?, ?B/s]
100% 16.6k/16.6k [00:00<00:00, 19.6MB/s]


In [3]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [4]:
## Cargamos la base
df = pd.read_csv('/content/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


## Procesamiento de la base

Vemos que la variable `Education Level` tiene varios valores similares escritos de manera distinta.

In [5]:
df["Education Level"].value_counts()

,count
Education Level,
Bachelor's Degree,2267
Master's Degree,1573
PhD,1368
Bachelor's,756
High School,448
Master's,288
phD,1


Para unificar estos valores, la recatogrizamos creando una nueva variable denominada `Education New`:

In [6]:
df["Education New"]=df["Education Level"].copy()

In [7]:
df.loc[df["Education Level"].str.contains("Bach", na=False), "Education New"] = "Bachelor's"
df.loc[df["Education Level"].str.contains("Mast", na=False), "Education New"] = "Master's"
df.loc[df["Education Level"].str.contains("hD", na=False), "Education New"] = "PhD"

Verificamos que con estos cambios, las categorías quedaron unificadas

In [8]:
df["Education New"].value_counts()

,count
Education New,
Bachelor's,3023
Master's,1861
PhD,1369
High School,448


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6698 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6698 non-null   float64
 1   Gender               6698 non-null   object 
 2   Education Level      6698 non-null   object 
 3   Job Title            6698 non-null   object 
 4   Years of Experience  6698 non-null   float64
 5   Salary               6698 non-null   float64
 6   Education New        6698 non-null   object 
dtypes: float64(3), object(4)
memory usage: 418.6+ KB


Borramos además los datos faltantes de la base

In [29]:
df=df.dropna()

Verificamos que ahora no quedan datos faltantes

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6698 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6698 non-null   float64
 1   Gender               6698 non-null   object 
 2   Education Level      6698 non-null   object 
 3   Job Title            6698 non-null   object 
 4   Years of Experience  6698 non-null   float64
 5   Salary               6698 non-null   float64
 6   Education New        6698 non-null   object 
dtypes: float64(3), object(4)
memory usage: 418.6+ KB


# Preguntas

## Pregunta 1

Realizar un test de hipótesis para ver si la proporción de mujeres es significativamente menor al 50%.

*Ayuda:* el atributo `.shape` devuelve la cantidad de filas y columnas de una base. Por lo tanto, tomando `df.shape[0]` se calcula la cantidad de filas de la base `df` y por lo tanto, la cantidad de individuos.
Para contar la cantidad de mujeres, por ejemplo, se puede recorrer todas las filas de "Gender" viendo en cada instancia si el valor coincide con "Female". Cuando se suman esos valores, se obtiene la cantidad de mujeres: `np.sum(df["Gender"]=="Female")`. El valor de referencia sería `value=0.5` y determinar la hipótesis alternativa correspondiente. Utilizar el `help` para ver cómo funciona el comando.


In [28]:
from statsmodels.stats.proportion import proportions_ztest

In [13]:
help(proportions_ztest)

Help on function proportions_ztest in module statsmodels.stats.proportion:

proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)
    Test for proportions based on normal (z) test
    
    Parameters
    ----------
    count : {int, array_like}
        the number of successes in nobs trials. If this is array_like, then
        the assumption is that this represents the number of successes for
        each independent sample
    nobs : {int, array_like}
        the number of trials or observations, with the same length as
        count.
    value : float, array_like or None, optional
        This is the value of the null hypothesis equal to the proportion in the
        case of a one sample test. In the case of a two-sample test, the
        null hypothesis is that prop[0] - prop[1] = value, where prop is the
        proportion in the two samples. If not provided value = 0 and the null
        is prop[0] = prop[1]
    alternative : str in ['two-sided', 'sma

In [30]:
np.sum(df["Gender"]=="Female")/df.shape[0]

0.44983577187220064

## Pregunta 2

Notemos que el porcentaje identificado como `Other` en la base es pequeño y no es un número que permita obtener conclusiones estadísticas pertinentes.

Realizar un test de hipótesis para ver si el salario medio de los hombres es **mayor** que el de las mujeres, según estos datos.

*Ayuda:* Se están evaluando medias, no porcentajes, por lo que se usa la función provista. Para comparar las medias, se deben pasar dos conjuntos de datos como argumento. Para identificar, por ejemplo, los datos de salarios sólo de las mujeres, se puede utilizar el siguiente comando: `df.Salary[df["Gender"]=="Female"]`

In [14]:
from statsmodels.stats.weightstats import ztest

In [15]:
help(ztest)

Help on function ztest in module statsmodels.stats.weightstats:

ztest(x1, x2=None, value=0, alternative='two-sided', usevar='pooled', ddof=1.0)
    test for mean based on normal distribution, one or two samples
    
    In the case of two samples, the samples are assumed to be independent.
    
    Parameters
    ----------
    x1 : array_like, 1-D or 2-D
        first of the two independent samples
    x2 : array_like, 1-D or 2-D
        second of the two independent samples
    value : float
        In the one sample case, value is the mean of x1 under the Null
        hypothesis.
        In the two sample case, value is the difference between mean of x1 and
        mean of x2 under the Null hypothesis. The test statistic is
        `x1_mean - x2_mean - value`.
    alternative : str
        The alternative hypothesis, H1, has to be one of the following
    
           'two-sided': H1: difference in means not equal to value (default)
           'larger' :   H1: difference in means la

## Pregunta 3

Realizar un test de ANOVA para ver si el salario medio cambia con los distintos niveles educativos. Utilizar la comparación múltiple para ver cuáles son diferentes entre sí

*Ayuda:* Los datos deben pasarse en una **lista**. Entonces, habría que definir previamente los datos de salario para cada nivel educativo. Para identificar el salario por cada nivel educativo, se puede usar la ayuda anterior reemplazando "Gender" por "Education New" y "Female" por "High School" para obtener los salarios de personas con título secundario completo. Para la comparación múltiple, se pueden usar directamente las columnas `df["Salary"]` (datos) y `df["Education New"]` (grupos).

In [16]:
from statsmodels.stats.oneway import anova_oneway
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [17]:
# Datos de salarios de individuos con título secundario completo
x1=
# Datos de salarios de individuos con título de grado completo
x2=
# Datos de salarios de individuos con título de maestría completo
x3=
# Datos de salarios de individuos con título de doctorado completo
x4=
print(anova_oneway([x1,x2,x3,x4]))
MC=MultiComparison()
print(MC.tukeyhsd())

SyntaxError: invalid syntax (<ipython-input-17-908668f83cbe>, line 2)

In [18]:
help(anova_oneway)
help(MultiComparison)

Help on function anova_oneway in module statsmodels.stats.oneway:

anova_oneway(data, groups=None, use_var='unequal', welch_correction=True, trim_frac=0)
    Oneway Anova
    
    This implements standard anova, Welch and Brown-Forsythe, and trimmed
    (Yuen) variants of those.
    
    Parameters
    ----------
    data : tuple of array_like or DataFrame or Series
        Data for k independent samples, with k >= 2.
        The data can be provided as a tuple or list of arrays or in long
        format with outcome observations in ``data`` and group membership in
        ``groups``.
    groups : ndarray or Series
        If data is in long format, then groups is needed as indicator to which
        group or sample and observations belongs.
    use_var : {"unequal", "equal" or "bf"}
        `use_var` specified how to treat heteroscedasticity, unequal variance,
        across samples. Three approaches are available
    
        "unequal" : Variances are not assumed to be equal across s

## Pregunta 4

Testear si la variable de años de experiencia tiene distribución normal y obtenga alguna conclusión. Hacer un histograma de la variable para comparar con los resultados del test.



In [19]:
import matplotlib.pyplot as plt
from scipy.stats import shapiro

In [20]:
help(shapiro)
help(plt.hist)

Help on function shapiro in module scipy.stats._morestats:

shapiro(x, *, axis=None, nan_policy='propagate', keepdims=False)
    Perform the Shapiro-Wilk test for normality.
    
    The Shapiro-Wilk test tests the null hypothesis that the
    data was drawn from a normal distribution.
    
    Parameters
    ----------
    x : array_like
        Array of sample data.
    axis : int or None, default: None
        If an int, the axis of the input along which to compute the statistic.
        The statistic of each axis-slice (e.g. row) of the input will appear in a
        corresponding element of the output.
        If ``None``, the input will be raveled before computing the statistic.
    nan_policy : {'propagate', 'omit', 'raise'}
        Defines how to handle input NaNs.
        
        - ``propagate``: if a NaN is present in the axis slice (e.g. row) along
          which the  statistic is computed, the corresponding entry of the output
          will be NaN.
        - ``omit``: Na

## Pregunta 5

Testear si hay correlación significativa entre las variables de años de experiencia y el salario. Hacer un gráfico de dispersión para contrastar con el resultado provisto por el test.



In [21]:
from scipy.stats import pearsonr

In [22]:
help(pearsonr)
help(plt.scatter)

Help on function pearsonr in module scipy.stats._stats_py:

pearsonr(x, y, *, alternative='two-sided', method=None)
    Pearson correlation coefficient and p-value for testing non-correlation.
    
    The Pearson correlation coefficient [1]_ measures the linear relationship
    between two datasets. Like other correlation
    coefficients, this one varies between -1 and +1 with 0 implying no
    correlation. Correlations of -1 or +1 imply an exact linear relationship.
    Positive correlations imply that as x increases, so does y. Negative
    correlations imply that as x increases, y decreases.
    
    This function also performs a test of the null hypothesis that the
    distributions underlying the samples are uncorrelated and normally
    distributed. (See Kowalski [3]_
    for a discussion of the effects of non-normality of the input on the
    distribution of the correlation coefficient.)
    The p-value roughly indicates the probability of an uncorrelated system
    producing 

In [23]:
from statsmodels.stats.weightstats import zconfint